# Connection Pooling

Little Lemon’s guests need to access the database for any booking or inquiry, for example, reading the menu. Little Lemon, therefore, needs to establish a connection between the Python and MySQL databases for every operation.  

Establishing a connection every time is resource intensive and it is affecting the performance of the Little Lemon application. 

To improve the performance of the application, Little Lemon needs to establish a pool of database connections to facilitate the guests’ inquiries to the database. 

Let's establish a connection by importing `MySQLConnectionPool` and creating a pool with two connections.

In [1]:
from mysql.connector.pooling import MySQLConnectionPool
from mysql.connector import Error

dbconfig = {
    "database":"little_lemon_db",
    "user" : "root",
    "password" : " "
}

try:
    pool = MySQLConnectionPool(pool_name = "pool_a",
                           pool_size = 2, #default is 5
                           **dbconfig)
    print("The connection pool is created with a name: ",pool.pool_name)
    print("The pool size is:",pool.pool_size)

except Error as er:
    print("Error code:", er.errno)
    print("Error message:", er.msg)

The connection pool is created with a name:  pool_a
The pool size is: 2


In [2]:
connection1 = pool.get_connection()

print("A user with connection id {} is connected to the database.".format(connection1.connection_id))

db_Info = connection1.get_server_info()
print("MySQL server version is:", db_Info)

# Create cursor object to communicate with entire MySQL database

print("Creating a cursor object.")
cursor = connection1.cursor()

A user with connection id 12 is connected to the database.
MySQL server version is: 8.0.35
Creating a cursor object.


# Stored Procedures


In [3]:
# This is the SQL query:

Stored_procedure_query = """
CREATE PROCEDURE PeakHours()
BEGIN
    SELECT HOUR(BookingSlot) AS 'Booking Hour', count(*) AS 'Number of Bookings' from bookings
    GROUP by BookingSlot
    ORDER by count(*) DESC;
END;
"""
# Invoke the execute module on the cursor:
cursor.execute(Stored_procedure_query)

# call the stored procedure:
cursor.callproc("PeakHours")

results = next(cursor.stored_results() )

dataset = results.fetchall()

cols = results.column_names
print(cols)

for data in dataset:
    print(data)

('Booking Hour', 'Number of Bookings')
(19, 2)
(15, 1)
(17, 1)
(18, 1)
(20, 1)


In [4]:
procedure_query = """
CREATE PROCEDURE GuestStatus()
BEGIN
    select 
        CONCAT(GuestFirstname, ' ', GuestLastName) AS "Guest FullName",
        CASE 
            WHEN Role IN ("Manager", "Assistant Manager") THEN "Ready to pay"
            WHEN Role = "Head Chef" THEN "Ready to serve"
            WHEN Role = "Assistant Chef" THEN "Preparing Order"
            ELSE "Order served"
        END AS "Order Status" 
    from employees
    LEFT JOIN bookings
    ON employees.EmployeeID = bookings.EmployeeID;
END;
"""
cursor.execute(procedure_query)

cursor.callproc("GuestStatus")

results = next(cursor.stored_results() )

dataset = results.fetchall()

cols = results.column_names
print(cols)

for data in dataset:
    print(data)

('Guest FullName', 'Order Status')
('Joakim Iversen', 'Ready to pay')
('Anna Iversen', 'Ready to pay')
('Hiroki Yamane', 'Ready to pay')
('Vanessa McCarthy', 'Ready to serve')
('Marcos Romero', 'Preparing Order')
('Diana Pinto', 'Order served')
(None, 'Order served')


Let's close the connection to return it back to the pool

In [5]:
connection1.close
print('Connection {} closed'.format(connection1.connection_id))


Connection 12 closed
